`curl 'https://tools.nycenet.edu/dashboard/api/data/dashboard/percentile_rank.php' -H 'User-Agent: Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:86.0) Gecko/20100101 Firefox/86.0' -H 'Accept: */*' -H 'Accept-Language: en-US,en;q=0.5' --compressed -H 'Content-Type: application/x-www-form-urlencoded; charset=UTF-8' -H 'X-Requested-With: XMLHttpRequest' -H 'Origin: https://tools.nycenet.edu' -H 'Connection: keep-alive' -H 'Referer: https://tools.nycenet.edu/dashboard/' -H 'Cookie: BIGipServerPR_TOOLS_NYCENET_EDU_POOL=779616778.20480.0000' -H 'DNT: 1' -H 'Sec-GPC: 1' -H 'Cache-Control: max-age=0' --data-raw 'dbn=02M418&report_type=HS&report_year=2020&CSRF_Token='`

In [1]:
#https://tools.nycenet.edu/dashboard/#dbn=02M418&report_type=HS&view=City

In [2]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from tqdm import tqdm

from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import random
import sys
import re
import pandas as pd
import numpy as np
import os
import time

Color-coding rules:
- Blue (Far Above Average): The school's result was in the 85th-100th percentile.
- Light Blue (Above Average): The school's result was in the 75th-84th percentile.
- Light Red (Below Average): The school's result was in the 16th-25th percentile.
- Red (Far Below Average): The school's result was in the 0-15th percentile.


In [3]:
def grab_table(driver, i,j,assertion, col_labels, row_labels):
    table_id = driver.find_element_by_id('mod-db-sa-metrics-tables-content-gridPanel-tbl-{}-{}-tPanel'.format(i,j))
    caption = table_id.find_element_by_tag_name('caption')
    assert(assertion in caption.text)
    rows_elem = table_id.find_elements_by_tag_name('tr')
    print(len(rows_elem))
    rows = []
    for i, row in enumerate(rows_elem):
        cells = [col.text for col in row.find_elements_by_tag_name('td')]
        zipped = dict(zip(col_labels, cells))
        if bool(zipped):
            rows.append(zipped)
    print(rows)
    return (caption.text.replace('\n', ' '), dict(zip(row_labels,rows)))

def rank_category(rank):
    if not rank.isdigit():
        return np.nan
    elif int(rank) >= 85:
        return "Far Above Average"
    elif int(rank) >= 75: 
        return "Above Average"
    elif int(rank) > 25:
        return "Average"
    elif int(rank) > 15:
        return "Below Average"
    else:
        return "Far Below Average"
    
class element_is_not_empty(object):
  """An expectation for checking that an element has a particular css class.

  locator - used to find the element
  returns the WebElement once it has the particular css class
  """
  def __init__(self, locator):
    self.locator = locator

  def __call__(self, driver):
    element = driver.find_element(*self.locator)   # Finding the referenced element
    if element.text:
        return element
    else:
        return False


In [4]:
foil = pd.read_csv('../data/foil/data.csv',header=[0,1])
dbns = list(foil[('School DBN', 'Unnamed: 0_level_1')])

In [5]:
def download_tables(dbn, report_type, table_ids=[(0,0,'4-Year Grad Rate'),(1,2,'College Readiness Index')]): 
    url = 'https://tools.nycenet.edu/dashboard/#dbn={}&report_type={}&view=City'.format(dbn, report_type)
    driver = webdriver.Firefox() 
    driver.get(url) 
    
    while True:
        try:
            wait = WebDriverWait(driver, random.randint(1,7))
            element = wait.until(element_is_not_empty((By.ID, 'mod-db-sa-metrics-tables-content-gridPanel-tbl-0-0-tPanel_2_2')))
            
            if element:
                select = Select(driver.find_element_by_id('dd-mod-db-sa-metrics-tables-content-view-dropdown'))
                select.select_by_visible_text('Percentiles')
                
                multiyr = driver.find_element_by_id('sa-metrics-tables-placeholder')
                x = multiyr.find_element_by_class_name('metric-tables-legend-block')
                row_label_elem = x.find_elements_by_class_name('row-item')
                row_labels = [row.text for row in row_label_elem]
                #    
                y = multiyr.find_element_by_id('sa-table-year-panel-0')
                col_label_elem = y.find_elements_by_tag_name('td')
                col_labels = [col.text for col in col_label_elem]
                #
                ## Get Graduation Rate
                tables = [grab_table(driver, a,b,c, col_labels, row_labels) for a,b,c in table_ids]
                print("got to this point....")
                driver.quit()
                
                return tables
            break
        except:
            print("Unexpected error:", sys.exc_info()[0])
            driver.refresh()

In [6]:
metrics = []

table_ids = [
    (0,0,'4-Year Grad Rate'),
    (1,2,'College Readiness Index'),
    (0,1,'Avg Regents Completion'),
    (1,1,'Avg Score: English (CC)'),
    (2,1,'Avg Score: Algebra I (CC)'),
    (0,2,'CCPCI (Advanced Courses)')
]

for dbn in tqdm(dbns):
    try:
        report_type = 'HS'
        tables = download_tables(dbn, report_type, table_ids=table_ids)
        for (table_name, table) in tables:
            df = pd.DataFrame(table)
            year = df.loc['2019'][["n", "School's Value", "City"]]
            row = {
                'dbn': dbn,
                'metric': table_name.split(',')[1],
                'n': year['n'],
                'school_value': year["School's Value"],
                'city_percentile_rank': year['City'],
                'city_percentile_rank_cat': rank_category(year['City'])
            }
            metrics.append(row)
    except:
        print(dbn)

  0%|          | 0/75 [00:00<?, ?it/s]

passed assertion
6
[{'2017': '88', '2018': '92', '2019': '94', '2020': '94'}, {'2017': '92%', '2018': '97%', '2019': '90%', '2020': '92%'}, {'2017': '57', '2018': '79', '2019': '43', '2020': 'N/A'}, {'2017': '68', '2018': '76', '2019': '63', '2020': 'N/A'}, {'2017': '77', '2018': '87', '2019': '68', '2020': 'N/A'}]
grabbed tables!!!
passed assertion
6
[{'2017': 'N/A', '2018': '92', '2019': '94', '2020': '94'}, {'2017': 'N/A', '2018': '89%', '2019': '83%', '2020': '87%'}, {'2017': 'N/A', '2018': 'N/A', '2019': '80', '2020': 'N/A'}, {'2017': 'N/A', '2018': '86', '2019': '77', '2020': 'N/A'}, {'2017': 'N/A', '2018': '93', '2019': '86', '2020': 'N/A'}]
grabbed tables!!!
passed assertion
6
[{'2017': 'N/A', '2018': 'N/A', '2019': 'N/A', '2020': 'N/A'}, {'2017': 'N/A', '2018': 'N/A', '2019': 'N/A', '2020': 'N/A'}, {'2017': 'N/A', '2018': 'N/A', '2019': 'N/A', '2020': 'N/A'}, {'2017': 'N/A', '2018': 'N/A', '2019': 'N/A', '2020': 'N/A'}, {'2017': 'N/A', '2018': 'N/A', '2019': 'N/A', '2020': 'N/

  1%|▏         | 1/75 [00:10<12:41, 10.29s/it]

Unexpected error: <class 'selenium.common.exceptions.TimeoutException'>
Unexpected error: <class 'selenium.common.exceptions.TimeoutException'>
passed assertion
6
[{'2017': '183', '2018': '128', '2019': '140', '2020': '158'}, {'2017': '97%', '2018': '96%', '2019': '100%', '2020': '96%'}, {'2017': '79', '2018': '74', '2019': '100', '2020': 'N/A'}, {'2017': '79', '2018': '71', '2019': '100', '2020': 'N/A'}, {'2017': '89', '2018': '85', '2019': '100', '2020': 'N/A'}]
grabbed tables!!!
passed assertion
6
[{'2017': 'N/A', '2018': '131', '2019': '140', '2020': '158'}, {'2017': 'N/A', '2018': '93%', '2019': '99%', '2020': '91%'}, {'2017': 'N/A', '2018': 'N/A', '2019': '95', '2020': 'N/A'}, {'2017': 'N/A', '2018': '90', '2019': '100', '2020': 'N/A'}, {'2017': 'N/A', '2018': '95', '2019': '99', '2020': 'N/A'}]
grabbed tables!!!
passed assertion
6
[{'2017': '160', '2018': '260', '2019': '279', '2020': 'N/A'}, {'2017': '92%', '2018': '90%', '2019': '85%', '2020': 'N/A'}, {'2017': '70', '2018': '7

  3%|▎         | 2/75 [00:31<20:04, 16.51s/it]

...

 97%|█████████▋| 73/75 [45:43<00:51, 25.72s/it]

Unexpected error: <class 'selenium.common.exceptions.TimeoutException'>
passed assertion
6
[{'2017': '235', '2018': '222', '2019': '185', '2020': '186'}, {'2017': '93%', '2018': '96%', '2019': '94%', '2020': '97%'}, {'2017': '53', '2018': '76', '2019': '57', '2020': 'N/A'}, {'2017': '69', '2018': '79', '2019': '71', '2020': 'N/A'}, {'2017': '79', '2018': '85', '2019': '77', '2020': 'N/A'}]
grabbed tables!!!
passed assertion
6
[{'2017': 'N/A', '2018': '222', '2019': '185', '2020': '186'}, {'2017': 'N/A', '2018': '84%', '2019': '84%', '2020': '91%'}, {'2017': 'N/A', '2018': 'N/A', '2019': '46', '2020': 'N/A'}, {'2017': 'N/A', '2018': '88', '2019': '79', '2020': 'N/A'}, {'2017': 'N/A', '2018': '90', '2019': '87', '2020': 'N/A'}]
grabbed tables!!!
passed assertion
6
[{'2017': '422', '2018': '377', '2019': '388', '2020': 'N/A'}, {'2017': '87%', '2018': '84%', '2019': '83%', '2020': 'N/A'}, {'2017': '51', '2018': '54', '2019': '50', '2020': 'N/A'}, {'2017': '87', '2018': '84', '2019': '83', 

 99%|█████████▊| 74/75 [45:59<00:22, 22.81s/it]

Unexpected error: <class 'selenium.common.exceptions.TimeoutException'>
Unexpected error: <class 'selenium.common.exceptions.TimeoutException'>
Unexpected error: <class 'selenium.common.exceptions.TimeoutException'>
Unexpected error: <class 'selenium.common.exceptions.TimeoutException'>
Unexpected error: <class 'selenium.common.exceptions.TimeoutException'>
Unexpected error: <class 'selenium.common.exceptions.TimeoutException'>
Unexpected error: <class 'selenium.common.exceptions.TimeoutException'>
Unexpected error: <class 'selenium.common.exceptions.TimeoutException'>
Unexpected error: <class 'selenium.common.exceptions.TimeoutException'>
Unexpected error: <class 'selenium.common.exceptions.TimeoutException'>
Unexpected error: <class 'selenium.common.exceptions.TimeoutException'>
Unexpected error: <class 'selenium.common.exceptions.TimeoutException'>
passed assertion
6
[{'2017': '88', '2018': '76', '2019': '55', '2020': '80'}, {'2017': '99%', '2018': '99%', '2019': '98%', '2020': '96%

100%|██████████| 75/75 [47:24<00:00, 37.93s/it]


In [7]:
metrics

[{'dbn': '01M450',
  'metric': ' 4-Year Grad Rate',
  'n': '94',
  'school_value': '90%',
  'city_percentile_rank': '68',
  'city_percentile_rank_cat': 'Average'},
 {'dbn': '01M450',
  'metric': ' College Readiness Index (no CAT)',
  'n': '94',
  'school_value': '83%',
  'city_percentile_rank': '86',
  'city_percentile_rank_cat': 'Far Above Average'},
 {'dbn': '01M450',
  'metric': ' Avg Regents Completion',
  'n': 'N/A',
  'school_value': 'N/A',
  'city_percentile_rank': 'N/A',
  'city_percentile_rank_cat': nan},
 {'dbn': '01M450',
  'metric': ' Avg Score: English (CC)',
  'n': '131',
  'school_value': '82',
  'city_percentile_rank': '85',
  'city_percentile_rank_cat': 'Far Above Average'},
 {'dbn': '01M450',
  'metric': ' Avg Score: Algebra I (CC)',
  'n': 'N/A',
  'school_value': 'N/A',
  'city_percentile_rank': 'N/A',
  'city_percentile_rank_cat': nan},
 {'dbn': '01M450',
  'metric': ' CCPCI (Advanced Courses)',
  'n': '94',
  'school_value': '88%',
  'city_percentile_rank': '88',


In [8]:
m = pd.DataFrame(metrics)

In [9]:
m[['dbn', 'metric', 'n','school_value', 'city_percentile_rank', 'city_percentile_rank_cat']]

,dbn,metric,n,school_value,city_percentile_rank,city_percentile_rank_cat
0,01M450,4-Year Grad Rate,94,90%,68,Average
1,01M450,College Readiness Index (no CAT),94,83%,86,Far Above Average
2,01M450,Avg Regents Completion,N/A,N/A,N/A,NaN
3,01M450,Avg Score: English (CC),131,82,85,Far Above Average
4,01M450,Avg Score: Algebra I (CC),N/A,N/A,N/A,NaN
...,...,...,...,...,...,...
445,30Q580,College Readiness Index (no CAT),55,98%,97,Far Above Average
446,30Q580,Avg Regents Completion,123,84%,89,Far Above Average
447,30Q580,Avg Score: English (CC),125,91,98,Far Above Average
448,30Q580,Avg Score: Algebra I (CC),44,70,75,Above Average


In [10]:
m[['dbn', 'metric', 'n','school_value', 'city_percentile_rank', 'city_percentile_rank_cat']].to_csv('../data/2019-metrics-rankings-full.csv', index=False)

In [16]:
metrics = m[['dbn','metric', 'city_percentile_rank_cat']].pivot(index='dbn',columns='metric',values='city_percentile_rank_cat').copy()

In [33]:
metrics.to_csv('../data/metrics_2019.csv')